<a href="https://colab.research.google.com/drive/1p9ZZ-c6iJ2jn8MpnPfB9gQ4KnUMz6pIV?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>


#	🔍 What is Hybrid Search?

Hybrid (Ensemble) Search is a technique that combines multiple search methods to improve retrieval performance. Typically, it combines traditional keyword-based search (like BM25) with semantic search (using embedding models). This approach can provide better results than either method alone, especially for queries that have both keyword-specific and semantic aspects.


# ❓ Hybrid-search RAG Implementation:

1. **Hybrid Retrieval:** We use the EnsembleRetriever to get relevant documents using both keyword-based and semantic search.
2. **Response Generation:** Using the retrieved context, we generate a final response to the original query.
3. **Hybrid Search Explanation:** We generate an explanation of how the hybrid search process might have improved the retrieval of relevant information.

# ⚙️ Setup

1. **[LLM](https://deepmind.google/technologies/gemini/pro/):** Google's free gemini-pro api endpoint ([Google's API Key](https://console.cloud.google.com/apis/credentials))
2. **[Vector Store](https://www.pinecone.io/learn/vector-database/):** [ChromaDB](https://www.trychroma.com/)
3. **[Embedding Model](https://qdrant.tech/articles/what-are-embeddings/):** [nomic-embed-text-v1.5](https://www.nomic.ai/blog/posts/nomic-embed-text-v1)
4. **[LLM Framework](https://python.langchain.com/v0.2/docs/introduction/):** LangChain
5. **[Huggingface API Key](https://huggingface.co/settings/tokens)**

# Install required libraries

In [ ]:
!pip install -q -U \
     Sentence-transformers==3.0.1 \
     langchain==0.2.11 \
     langchain-google-genai==1.0.7 \
     langchain-chroma==0.1.2 \
     langchain-community==0.2.10 \
     langchain-huggingface==0.0.3 \
     einops==0.8.0 \
     rank_bm25==0.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 12.8 MB/s et

# Import related libraries related to Langchain, HuggingfaceEmbedding

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import WebBaseLoader
from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [ ]:
import getpass
import os

# Provide Google API Key. You can create Google API key at following link

[Google Gemini-Pro API Creation Link](https://console.cloud.google.com/apis/credentials)

[YouTube Video](https://www.youtube.com/watch?v=ZHX7zxvDfoc)



In [ ]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass()

··········


# Provide Huggingface API Key. You can create Huggingface API key at following lin

[Higgingface API Creation Link](https://huggingface.co/settings/tokens)




In [ ]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


# Step 1: Load and preprocess data

In [ ]:
def load_and_process_data(url):
    # Load data from web
    loader = WebBaseLoader(url)
    data = loader.load()

    # Split text into chunks (Experiment with Chunk Size and Chunk Overlap to get optimal chunking)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(data)

    return chunks

# Step 2: Create vector store and BM25 retriever

In [ ]:
def create_retrievers(chunks):
    embeddings = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1.5", model_kwargs = {'trust_remote_code': True})
    vectorstore = Chroma.from_documents(chunks, embeddings)
    bm25_retriever = BM25Retriever.from_documents(chunks)
    bm25_retriever.k = 5  # Set number of documents to retrieve

    ensemble_retriever = EnsembleRetriever(
        retrievers=[bm25_retriever, vectorstore.as_retriever(search_kwargs={"k": 5})],
        weights=[0.5, 0.5]
    )

    return ensemble_retriever

# Step 3: Hybrid-search RAG related code

1. **Hybrid Retrieval:** We use the EnsembleRetriever to get relevant documents using both keyword-based and semantic search.
2. **Response Generation:** Using the retrieved context, we generate a final response to the original query.
3. **Hybrid Search Explanation:** We generate an explanation of how the hybrid search process might have improved the retrieval of relevant information.

In [ ]:
def hybrid_search_rag(query, ensemble_retriever, llm):
    # Hybrid retrieval
    retrieved_docs = ensemble_retriever.invoke(query)
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])

    # Generate response
    response_prompt = ChatPromptTemplate.from_template(
        "You are an AI assistant tasked with answering questions based on the provided context. "
        "The context contains information retrieved using a hybrid search method combining keyword-based and semantic search. "
        "Please provide a comprehensive answer to the question, using the context when relevant "
        "and your general knowledge when necessary.\n\n"
        "Context:\n{context}\n\n"
        "Question: {query}\n"
        "Answer:"
    )
    try:
        response = llm.invoke(response_prompt.format(context=context, query=query))
        final_answer = response.content
    except Exception as e:
        print(f"Error generating response: {e}")
        final_answer = "I apologize, but I encountered an error while generating the response."

    # Generate explanation of hybrid search process
    explanation_prompt = ChatPromptTemplate.from_template(
        "Explain how the hybrid search process, combining keyword-based and semantic search, "
        "might have improved the retrieval of relevant information for answering the given query. "
        "Consider the potential benefits of this approach compared to using only one search method.\n\n"
        "Query: {query}\n"
        "Explanation:"
    )
    try:
        explanation = llm.invoke(explanation_prompt.format(query=query))
        hybrid_search_explanation = explanation.content
    except Exception as e:
        print(f"Error generating explanation: {e}")
        hybrid_search_explanation = "Unable to generate explanation due to an error."

    return {
        "query": query,
        "final_answer": final_answer,
        "hybrid_search_explanation": hybrid_search_explanation,
        "retrieved_context": context
    }

# Step 4: Create chunk of web data to Chroma Vector Store

In [ ]:
# Initialize the gemini-pro language model with specified settings (Change temeprature  and other parameters as per your requirement)
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3, safety_settings={
          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        },)

# Load and process data
url = "https://en.wikipedia.org/wiki/Artificial_intelligence"
chunks = load_and_process_data(url)

# Create hybrid_results retriever
ensemble_retriever  = create_retrievers(chunks)

In [ ]:
# Example queries
queries = [
        "What are the main applications of artificial intelligence in healthcare?"
    ]

# Run Hybrid-search RAG for each query
for query in queries:
  print(f"\nQuery: {query}")
  result = hybrid_search_rag(query, ensemble_retriever, llm)
  print("Final Answer:")
  print(result["final_answer"])
  print("\nHybrid Search Explanation:")
  print(result["hybrid_search_explanation"])
  print("\nRetrieved Context (first 300 characters):")
  print(result["retrieved_context"][:300] + "...")


Query: What are the main applications of artificial intelligence in healthcare?
Final Answer:
The main applications of artificial intelligence in healthcare include increasing patient care and quality of life. AI can be used to more accurately diagnose and treat patients, as well as to assist with tasks such as medical research and drug discovery.

Hybrid Search Explanation:
**Hybrid Search Process**

The hybrid search process combines keyword-based and semantic search to enhance the retrieval of relevant information.

**Keyword-Based Search:**

* Identifies documents containing specific keywords from the query.
* Provides a broad set of results, including both relevant and irrelevant documents.

**Semantic Search:**

* Analyzes the meaning and context of the query.
* Identifies documents that are semantically related to the query, even if they do not contain exact keywords.

**Benefits of Hybrid Search:**

**Improved Relevance:**

* Semantic search complements keyword-based search by